In [40]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import json

In [19]:
root_path = !pwd
root_path = str(root_path[0])

In [41]:
filepath = f"{root_path}/playlist_data/sampledata/"
filename = 'mpd.slice.0-999.json'

In [42]:
fpath_name = f"{filepath}{filename}"
print(fpath_name)

/home/ernek/Main/Erdos/song_recommender/playlist_data/sampledata/mpd.slice.0-999.json


In [127]:
with open(fpath_name) as data_file:    
    data = json.load(data_file)  

print("File keys: ", data.keys())
print(list(data['playlists'][0].keys()))
print(data['playlists'][0]['tracks'][0].keys())
# df = pd.json_normalize(data, 'locations', ['date', 'number', 'name'], 
#                     record_prefix='locations_')
# print(df)
num_keys_old = 0
for index, playlist in enumerate(data['playlists']):
    num_keys =  len(playlist.keys())
    if index == 0:
        num_key_old = num_keys
        continue
    if num_keys > num_key_old:
        keys = playlist.keys()
    num_key_old = num_keys

keys = list(keys)
keys.remove('tracks')
print(keys)


File keys:  dict_keys(['info', 'playlists'])
['name', 'collaborative', 'pid', 'modified_at', 'num_tracks', 'num_albums', 'num_followers', 'tracks', 'num_edits', 'duration_ms', 'num_artists']
dict_keys(['pos', 'artist_name', 'track_uri', 'artist_uri', 'track_name', 'album_uri', 'duration_ms', 'album_name'])
['name', 'collaborative', 'pid', 'modified_at', 'num_tracks', 'num_albums', 'num_followers', 'num_edits', 'duration_ms', 'num_artists', 'description']


In [130]:
music_df = pd.json_normalize(data['playlists'],  meta = keys, meta_prefix = 'playlist_', errors='ignore', record_path=['tracks'],record_prefix = 'track_')

In [137]:
music_df[music_df['playlist_pid'] == 0]

,track_pos,track_artist_name,track_track_uri,track_artist_uri,track_track_name,track_album_uri,track_duration_ms,track_album_name,playlist_name,playlist_collaborative,playlist_pid,playlist_modified_at,playlist_num_tracks,playlist_num_albums,playlist_num_followers,playlist_num_edits,playlist_duration_ms,playlist_num_artists,playlist_description
0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
1,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
2,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
3,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
4,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
5,5,Usher,spotify:track:0XUfyU2QviPAs6bxSpXYG4,spotify:artist:23zg3TcAtWQy7J6upgbUnj,Yeah!,spotify:album:0vO0b1AvY49CPQyVisJLj0,250373,Confessions,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
6,6,Usher,spotify:track:68vgtRHr7iZHpzGpon6Jlo,spotify:artist:23zg3TcAtWQy7J6upgbUnj,My Boo,spotify:album:1RM6MGv6bcl6NrAG8PGoZk,223440,Confessions,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
7,7,The Pussycat Dolls,spotify:track:3BxWKCI06eQ5Od8TY2JBeA,spotify:artist:6wPhSqRtPu1UhRCDX5yaDJ,Buttons,spotify:album:5x8e8UcCeOgrOzSnDGuPye,225560,PCD,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
8,8,Destiny's Child,spotify:track:7H6ev70Weq6DdpZyyTmUXk,spotify:artist:1Y8cdNmUJH7yBTd9yOvr5i,Say My Name,spotify:album:283NWqNsCA9GwVHrJk59CG,271333,The Writing's On The Wall,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
9,9,OutKast,spotify:track:2PpruBYCo4H7WOBJ7Q2EwM,spotify:artist:1G9G7WwrXka3Z1r7aIDjI7,Hey Ya! - Radio Mix / Club Mix,spotify:album:1UsmQ3bpJTyK6ygoOOjG1r,235213,Speakerboxxx/The Love Below,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN


,track_pos,track_artist_name,track_track_uri,track_artist_uri,track_track_name,track_album_uri,track_duration_ms,track_album_name,playlist_name,playlist_collaborative,playlist_pid,playlist_modified_at,playlist_num_tracks,playlist_num_albums,playlist_num_followers,playlist_num_edits,playlist_duration_ms,playlist_num_artists,playlist_description
0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
1,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
2,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
3,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
4,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67459,0,Pvris,spotify:track:3rUTBx7gmn1IAAwsrjtnO0,spotify:artist:6oFs3qk4VepIVFdoD4jmsy,Mind over Matter (Acoustic),spotify:album:33oxbqzo7poSO2IbSMqrPS,203649,Acoustic,thinking of you,false,999,1507766400,44,43,1,34,9065801,37,NaN
67460,1,Ingrid Michaelson,spotify:track:6zYqyOJuyXDOvWJwKkZ8tg,spotify:artist:2vm8GdHyrJh2O2MfbQFYG0,You and I,spotify:album:7o2VLivg95UduHjTMTIEIf,148400,Be OK,thinking of you,false,999,1507766400,44,43,1,34,9065801,37,NaN
67461,2,Troye Sivan,spotify:track:0nRuVPZx2iA3KWR4xSzryl,spotify:artist:3WGpXCj9YhhfX11TToZcXP,TALK ME DOWN,spotify:album:5ouTDazE4LF9bVJPx1nlgW,237464,Blue Neighbourhood,thinking of you,false,999,1507766400,44,43,1,34,9065801,37,NaN
67462,3,City and Colour,spotify:track:2pbxqEYiXJTvFsybGGgSAi,spotify:artist:74gcBzlQza1bSfob90yRhR,Un-Thinkable - I'm Ready,spotify:album:2wCLwVcPgBG1UHszyqV6EA,297133,"Covers, Pt. 3",thinking of you,false,999,1507766400,44,43,1,34,9065801,37,NaN
